In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os

In [ ]:
def crear_buffer(lat, lon, radio_km, resolution=32):
    punto = Point(lon, lat)
    return punto.buffer((radio_km + 1.2) / 111, resolution=resolution)

def filtrar_datos(df, buffer):
    # df = df.copy()
        # Convertir las columnas de longitud y latitud a float
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
        # Filtrar los datos dentro del buffer
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))
    return gdf[gdf.geometry.within(buffer)]


In [ ]:

# Ubicaciones de interés y radio
ubicaciones = {
     'Asuncion': (-25.301478, -57.536707),
    #'Central': (-25.5628115, -57.5079912),
    # 'Ñeembucu': (-26.648707, -58.107135),
    # 'Concepcion': (-23.434573, -57.434246)
}
radio_km = 25

# Rutas y años de interés
ruta_base = '/content/drive/MyDrive/2.TPInteligenciaArtificial/Organizado/Anuales/'
anios = [
      '2016',
    #      '2017',
    #       '2018',
    #       '2019',
          # '2020',
          #'2021',
          # '2022'
          ]
# anios = [ '2020','2021', '2022']


In [ ]:
 for anio in anios:
  ruta_archivo_anual = os.path.join(ruta_base, f'{anio}.csv')
  df_anual=pd.read_csv(ruta_archivo_anual)
  if 'timestamp' in df_anual.columns:
    df_anual = pd.read_csv(ruta_archivo_anual,parse_dates=True,on_bad_lines='skip')
  else:
    df_anual = pd.read_csv(ruta_archivo_anual, parse_dates=['LightningTime'], date_format='%Y-%m-%dT%H:%M:%S.%f', on_bad_lines='skip')
    df_anual['timestamp']=df_anual['LightningTime']
    df_anual.drop(columns=['LightningTime'],inplace=True)
  df_anual.set_index('timestamp',inplace=True)

  # print(f'{anio}',df_anual['timestamp'])
  for ubicacion, coords in ubicaciones.items():
      buffer = crear_buffer(coords[0], coords[1], radio_km)
      df_filtrado = filtrar_datos(df_anual, buffer)

      archivo_salida = os.path.join(ruta_base, f'{anio}_filtrado_{ubicacion}.csv')
      df_filtrado.drop(columns=['geometry']).to_csv(archivo_salida, index=True)
      # df_filtrado.drop(columns=['Hright',])
      print(f"Archivo filtrado para {ubicacion} del año {anio} guardado en {archivo_salida}")
      print(df_filtrado)

Archivo filtrado para Asuncion del año 2016 guardado en /content/drive/MyDrive/2.TPInteligenciaArtificial/Organizado/Anuales/2016_filtrado_Asuncion.csv
                               type   latitude  longitude  peakcurrent  \
timestamp                                                                
2016-01-01 00:33:35.956890106    40 -25.274600 -57.539900       -12448   
2016-01-01 00:33:36.154273987     0 -25.331540 -57.462260        74825   
2016-01-01 00:49:14.815330029    40 -25.121400 -57.564400        20092   
2016-01-01 01:02:11.518081903    40 -25.204900 -57.573600        15486   
2016-01-01 01:14:21.134871006    40 -25.187500 -57.679200       -25255   
...                             ...        ...        ...          ...   
2016-12-31 11:42:54.062751532     0 -25.136776 -57.687196       -24065   
2016-12-31 11:42:54.099242926    40 -25.169600 -57.701800       -13894   
2016-12-31 11:42:54.265235662     0 -25.128071 -57.609894       -19307   
2016-12-31 11:43:26.413749456     